# RIA - a IA retratista

### A RIA é uma IA especializada em gerar retratos do rosto dos modelos que autorizam a produção. Contudo, a RIA é uma artista incompreendida (nem mesmo ela se compreende) e não salva as artes que produz, por que julga não estarem perfeitas.

## 🧠 Como usar a RIA:
Assim que a câmera for aberta, clique na janela da câmera.

⚠️ É importante manter a janela da câmera ativa (selecionada) para que os comandos funcionem corretamente.

### Escolha o estilo do retrato pressionando uma das teclas:

### 1 – Lápis de cor
### 2 – Tinta à base d’água
### 3 – Tinta a óleo

Após escolher, aguarde alguns segundos para que a arte seja gerada.

### Depois de ter o autoretrato, digite **c** para outro retrato

### Para encerrar a RIA, digite **q**.

In [ ]:
# Importação das bibliotecas necessárias
import cv2
import mediapipe as mp

import urllib.request

# URLs dos modelos de estilização de rosto
urls = {
    "face_stylizer_lapis.task": "https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_color_sketch.task",
    "face_stylizer_tinta.task": "https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_color_ink.task",
    "face_stylizer_oleo.task": "https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_oil_painting.task",
}

# Faz o download dos modelos, se ainda não estiverem disponíveis localmente
for name, url in urls.items():
    urllib.request.urlretrieve(url, name)

# Importa os módulos do MediaPipe Tasks
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Define o tamanho desejado para o retrato final
DESIRED_HEIGHT = 680
DESIRED_WIDTH = 680

# Variáveis auxiliares
frame = None
base_options = None
options = None

# Início do loop principal da aplicação
while True:
    # Define modelo padrão (pode ser substituído com a tecla 1, 2 ou 3 depois)
    base_options = python.BaseOptions(model_asset_path='face_stylizer_lapis.task')

    # Inicia a captura da webcam
    cap = cv2.VideoCapture(0) 

    while True:
        encerra = False # Controla a saída do loop interno (escolha de estilo)
        
        # Captura um frame da webcam
        ret, frame = cap.read()
        if not ret:
            print("Falha em obter o frame.")
            break
        
        # Exibe o vídeo da câmera ao vivo
        cv2.imshow('Camera Feed', frame)
        
        # Verifica se uma tecla foi pressionada para mudar o estilo do retrato
        if cv2.waitKey(10) & 0xFF == ord('1'):
            base_options = python.BaseOptions(model_asset_path='face_stylizer_lapis.task')
            encerra = True
        elif cv2.waitKey(10) & 0xFF == ord('2'):
            base_options = python.BaseOptions(model_asset_path='face_stylizer_tinta.task')
            encerra = True
        elif cv2.waitKey(10) & 0xFF == ord('3'):
            base_options = python.BaseOptions(model_asset_path='face_stylizer_oleo.task')
            encerra = True
        
        # Sai do loop de câmera para aplicar o estilo selecionado
        if encerra:
            cap.release()               # Encerra a captura de vídeo
            cv2.destroyAllWindows()     # Fecha a janela da câmera
            cv2.waitKey(1)              # Dá tempo para a janela fechar corretamente
            break
    
    # Configura o estilizador com o modelo escolhido
    options = vision.FaceStylizerOptions(base_options=base_options)
    
    # Cria o estilizador com base nas opções definidas
    with vision.FaceStylizer.create_from_options(options) as stylizer:
        
        # Converte o frame capturado em imagem do MediaPipe
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # Aplica o estilo artístico ao rosto
        stylized_image = stylizer.stylize(mp_image)

        if not stylized_image is None:
            # Converte imagem estilizada para RGB e redimensiona para exibição
            rgb_stylized_image = cv2.cvtColor(stylized_image.numpy_view(), cv2.COLOR_BGR2RGB)
            rgb_stylized_image_resize = cv2.resize(rgb_stylized_image, (DESIRED_WIDTH, DESIRED_WIDTH))

            # Exibe o retrato gerado
            cv2.imshow('Retrato', rgb_stylized_image_resize)
            
            # Aguarda uma tecla: 'c' para novo retrato ou 'q' para sair
            if cv2.waitKey(0) & 0xFF == ord('c'):
                cap.release()
                cv2.destroyAllWindows()
                cv2.waitKey(1)

            elif cv2.waitKey(0) & 0xFF == ord('q'):
                break
            
# Libera recursos e fecha janelas restantes
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)